# COMP 527: Implementing the k-means clustering algorithm

> In the assignment, you are required to cluster words belonging to four categories: animals, countries, fruits and veggies. The words are arranged into four different files. The first entry in each line is a word followed by 300 features (word embedding) describing the meaning of that word.

## Implementation
 
> (1) Implement the k-means clustering algorithm with Euclidean distance to cluster the instances into k clusters. (30 marks)

In [1]:
import numpy as np
import scipy
import matplotlib.pyplot as plt

In [41]:
"""Read in data."""

categories = ['animals', 'countries', 'fruits', 'veggies']

data = {}
words = []
for category in categories:
    data[category] = open(category).read().split('\n')[:-1]
    print(category, len(data[category]))
    
    for word_data in data[category]:
        split = word_data.split(' ')
        word = split[0]
        raw_list = split[1:]
        
        floats = []
        for x_string in vector_raw:
            floats.append(float(x_string))
            
        vector = np.array(floats)
        
        words.append(
            {
            'name':word,
            'vector':vector,
            'category':category
            }
        )

animals 50
countries 161
fruits 58
veggies 60


In [44]:
type(words[0]['vector'])

numpy.ndarray

In [ ]:
class Metric():
    
    def __init__():
        pass
    
    def check_welldefined():
        pass
    
    def distance(self, x, y):
        

class KMeans():
    
    def __init__(self, k = 5, data = words, metric = Euclidean):
        """
        Initialize KMeans Model.
        
        Args:
            k (int): number of clusters to divide data into.
            data (list): list of dicts which must each include
                the keys 'name' (string) and 'vector' (np.ndarray).
        """
        
        self.k = k
        self.data = data
        
    

In [37]:
type(words[0]['vector'][0])

float

In [31]:
number = '0.53533'

In [33]:
float(number)

0.53533

In [20]:
len(data['animals'])

50

In [13]:
]


animal = open('animals').read().split('\n')[:-1]

In [15]:
for a in animals:
    

50

In [8]:
animals = animals.split('\n')

AttributeError: 'list' object has no attribute 'split'

In [ ]:
a

## Initial Computation

> (2) Vary the value of k from 1 to 10 and compute the precision, recall, and F-score for each set of clusters. Plot k in the horizontal axis and precision, recall and F-score in the vertical axis in the same plot. (10 marks)

## Normalize

> (3) Now re-run the k-means clustering algorithm you implemented in part (1) but normalise each feature vector to unit L2 length before computing Euclidean distances. Vary the value of k from 1 to 10 and compute the precision, recall, and F-score for each set of clusters. Plot k in the horizontal axis and precision, recall and F-score in the vertical axis in the same plot. (10 marks)

## Manhattan Distance

> (4) Now re-run the k-means clustering algorithm you implemented in part (1) but this time use Manhattan distance over the unnormalised feature vectors. Vary the value of k from 1 to 10
and compute the precision, recall, and F-score for each set of clusters. Plot k in the horizontal axis and precision, recall and F-score in the vertical axis in the same plot. (10 marks)

## Normalized Manhattan Distance

> (5) Now re-run the k-means clustering algorithm you implemented in part (1) but this time use Manhattan distance with L2 normalised feature vectors. Vary the value of k from 1 to 10 and
compute the precision, recall, and F-score for each set of clusters. Plot k in the horizontal axis and precision, recall and F-score in the vertical axis in the same plot. (10 marks)

##  Cosine Similarity

> (6) Now re-run the k-means clustering algorithm you implemented in part (1) but this time use cosine similarity as the distance (similarity) measure.Vary the value of k from 1 to 10 andcompute the precision, recall, and F-score for each set of clusters. Plot k in the horizontal axis and precision, recall and F-score in the vertical axis in the same plot. (10 marks)

## Compare and Discuss

> (7) Comparing the different clusterings you obtained in (2)-(6) discuss what is the best setting for k-means clustering for this dataset. (20 marks)